In [5]:
# Author: Davide Aloi
# Description: Opens each joystick scan (and passive mobilisation) and checks if there were
# voxels that survive thresholding (p=0.001 --> t = 3.13)
# This is done for 3 regions of interests (M1, sma and th) that were transformed from the MNI
# space to the patient's volume space.

import nilearn
from nilearn import image
import glob
import os
import numpy as np
from nilearn import reporting

path_rois = 'C:\\Users\\davide\\Documents\\GitHub\\raindrop_analyses_fmri_eeg\\fmri\\rois\\'
m1 = image.load_img(glob.glob(path_rois + 'wm1.nii')[0]) # m1 roi in the native space
sma = image.load_img(glob.glob(path_rois + 'wsma.nii')[0]) # sma roi in the native space
th = image.load_img(glob.glob(path_rois + 'wth.nii')[0]) # th roi in the native space
rois = [m1, sma, th]
roi_name = ['m1', 'sma', 'th']

threshold = 3.13 # t threshold for p 0.001 uncorrected

In [6]:
def clean_table(table, n_vols):
        # n_vols is the number of volumes of the functional scan.
        # this is used to calculate the degrees of freedom to retrieve the
        # p value for each t suprathreshold. However, this is not entirely correct
        # because DF should be equal to: number of scans - rank design matrix.

        import scipy.stats
        table['X'] = table['X'].round().astype(int)
        table['Y'] = table['Y'].round().astype(int)
        table['Z'] = table['Z'].round().astype(int)
        
        # find p-value and add it to table
        p_vals = scipy.stats.t.sf(abs(table['Peak Stat']), df=n_vols)
        table['p_uncor'] = p_vals
        table.sort_values('p_uncor', axis=0, ascending=True, inplace=True)
        return(table)

In [7]:
# Results Joystick task
# creating list with SPM_t results for the joystick task
path_res_p0 = 'D:\\Raindrop_data\\p01\\'
path_joystick_results = glob.glob(path_res_p0 + 'p01_*\\day*\\fmri_data\\JOYSTICK*\\nifti\\glm_first_level\\spmT_0002.nii')
path_joystick_scans = glob.glob(path_res_p0 + 'p01_*\\day*\\fmri_data\\JOYSTICK*\\nifti\\rj*.nii')

# looping through spm_t maps applying the 3 rois to each spm_t
# then checking if suprathreshold voxels are present
for n, spm_t in enumerate(path_joystick_results):
    print('Opening ' + spm_t)

    # Loading spmt_0002.nii
    spm_t_img = image.load_img(spm_t)

    # masking results for each roi
    for n_roi, roi in enumerate(rois):
        
        print('Applying ROI: ' + roi_name[n_roi])
        roi_resampled = image.resample_to_img(roi, spm_t_img, interpolation = 'nearest')

        # masking
        spm_t_img_masked = np.where(roi_resampled.get_fdata() == 1, spm_t_img.get_fdata(), 0)

        # counting suprathreshold voxels
        supr_voxels = (spm_t_img_masked.flatten() > threshold).sum()
        print('number of significant voxels: ' + str(supr_voxels))

        # If there are suprathreshold voxels --> table (which is almost the same as the one
        # you get from spm)
        if supr_voxels > 0:
            table = reporting.get_clusters_table(image.new_img_like(spm_t_img, spm_t_img_masked), threshold, 1, min_distance = 8)
            n_vols = image.load_img(path_joystick_scans[n]).get_fdata().shape[3]
            table = clean_table(table, n_vols) # create spm like table
            # saving table
            # nb this fname depends on where you have the files. You'll probably have to change it
            fname = path_joystick_scans[n].split('\\')[3] + '_'+ path_joystick_scans[n].split('\\')[4] + '_' + path_joystick_scans[n].split('\\')[6] +  '_' + roi_name[n_roi] + '.csv'
            print(table)
            table.to_csv(fname)
            #print(table.round(3))

# Results can be verified by loading the spm.mat file and applying a ROI
# NB the tables resulting from get_clusters_table might differ slighly from SPM ones
# as SPM outputs the first 3 local maxima only.

Opening D:\Raindrop_data\p01\p01_w02\day01\fmri_data\JOYSTICK_BASELINE_0015\nifti\glm_first_level\spmT_0002.nii
Applying ROI: m1
number of significant voxels: 0
Applying ROI: sma
number of significant voxels: 0
Applying ROI: th
number of significant voxels: 0
Opening D:\Raindrop_data\p01\p01_w02\day01\fmri_data\JOYSTICK_POST_0019\nifti\glm_first_level\spmT_0002.nii
Applying ROI: m1
number of significant voxels: 0
Applying ROI: sma
number of significant voxels: 0
Applying ROI: th
number of significant voxels: 0
Opening D:\Raindrop_data\p01\p01_w02\day05\fmri_data\JOYSTICK_BASELINE_0005\nifti\glm_first_level\spmT_0002.nii
Applying ROI: m1
number of significant voxels: 0
Applying ROI: sma
number of significant voxels: 0
Applying ROI: th
number of significant voxels: 0
Opening D:\Raindrop_data\p01\p01_w02\day05\fmri_data\JOYSTICK_POST_0010\nifti\glm_first_level\spmT_0002.nii
Applying ROI: m1
number of significant voxels: 0
Applying ROI: sma
number of significant voxels: 0
Applying ROI: th


In [8]:
# Results Passive Mobilisation: SAME AS ABOVE BUT WITHOUT COMMENTS
# creating list with SPM_t results for the joystick task

path_res_p0 = 'D:\\Raindrop_data\\p01\\'
path_passive_results = glob.glob(path_res_p0 + 'p01_*\\day*\\fmri_data\\PASSIVE*\\nifti\\glm_first_level\\spmT_0002.nii')
path_passive_scans = glob.glob(path_res_p0 + 'p01_*\\day*\\fmri_data\\PASSIVE*\\nifti\\rp*.nii')

for n, spm_t in enumerate(path_passive_results):
    print('Opening ' + spm_t)
    spm_t_img = image.load_img(spm_t)

    for n_roi, roi in enumerate(rois):

        print('Applying ROI: ' + roi_name[n_roi])
        roi_resampled = image.resample_to_img(roi, spm_t_img, interpolation = 'nearest')
        spm_t_img_masked = np.where(roi_resampled.get_fdata() == 1, spm_t_img.get_fdata(), 0)
        
        supr_voxels = (spm_t_img_masked.flatten() > threshold).sum()
        print('number of significant voxels: ' + str(supr_voxels))

        if supr_voxels > 0:
            table = reporting.get_clusters_table(image.new_img_like(spm_t_img, spm_t_img_masked), threshold, 1, min_distance = 8)
            n_vols = image.load_img(path_passive_scans[n]).get_fdata().shape[3]
            table = clean_table(table, n_vols)

            # nb this fname depends on where you have the files. You'll probably have to change it
            fname = path_passive_scans[n].split('\\')[3] + '_'+ path_passive_scans[n].split('\\')[4] + '_' + path_passive_scans[n].split('\\')[6] + '_' + roi_name[n_roi] + '.csv'
            print(table)
            table.to_csv(fname)
            print(table)


Opening D:\Raindrop_data\p01\p01_w02\day01\fmri_data\PASSIVE_MOV_DURING_TDCS_0018\nifti\glm_first_level\spmT_0002.nii
Applying ROI: m1
number of significant voxels: 0
Applying ROI: sma
number of significant voxels: 0
Applying ROI: th
number of significant voxels: 0
Opening D:\Raindrop_data\p01\p01_w02\day05\fmri_data\PASSIVE_MOV_DURING_TDCS_0009\nifti\glm_first_level\spmT_0002.nii
Applying ROI: m1
number of significant voxels: 0
Applying ROI: sma
number of significant voxels: 2
   Cluster ID  X  Y   Z  Peak Stat  Cluster Size (mm3)   p_uncor
0           1 -1  0  37   3.184201                  31  0.000759
   Cluster ID  X  Y   Z  Peak Stat  Cluster Size (mm3)   p_uncor
0           1 -1  0  37   3.184201                  31  0.000759
Applying ROI: th
number of significant voxels: 0
Opening D:\Raindrop_data\p01\p01_w04\day01\fmri_data\PASSIVE_MOV_DURING_TDCS_0008\nifti\glm_first_level\spmT_0002.nii
Applying ROI: m1
number of significant voxels: 0
Applying ROI: sma
number of significant v